In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
combi = pd.concat([train.drop('target', axis=1), test], axis=0)

In [9]:
# Unnecessary

# from sklearn.preprocessing import LabelEncoder

# for i in combi.select_dtypes(exclude=[np.number]).columns:
#     lbl = LabelEncoder()
#     lbl.fit(combi[i])
#     train[i] = lbl.transform(train[i])
#     test[i] = lbl.transform(test[i])

In [6]:
# Too much data for my computer, so we'll take a sample
train = train.sample(frac=1.0)
train_sample = train.sample(20000)

train_sample = train_sample.head(10000)
test = train_sample.tail(10000)

x = train_sample.drop('target', axis=1)
y = train_sample['target']

In [8]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, KFold
kf = KFold(n_splits=3, shuffle=True)

clf = XGBClassifier()
params = {'max_depth': [2],
         'n_estimators': np.arange(80,130,1),
         'reg_lambda': np.arange(1.4, 1.61, .1)}

grid = GridSearchCV(clf, param_grid=params, cv=kf, scoring='roc_auc').fit(x, y)
clf = grid.best_estimator_

cv = cross_val_score(clf, x, y, cv=kf, scoring='roc_auc')
# Mean score: 0.619952716139
print(clf, '\n')
print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=107,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1.3999999999999999, scale_pos_weight=1,
       seed=0, silent=True, subsample=1) 

Mean score: 0.610121851248
Std Dev:    0.00829439515399


In [ ]:
def absolute_correlations(col, df=data):
    #absolute_values = np.abs(df[col])
    corrs = pd.DataFrame(df.select_dtypes(include=[np.number]).corrwith(df[col]), columns=['correlation'])
    corrs['absol'] = np.abs(corrs['correlation'])
    return corrs.sort_values('absol', ascending=False).drop('absol', axis=1).tail(len(corrs)-1)

absolute_correlations('target')